In [3]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ollama
import pandas as pd
import time
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import subprocess

# Step 1: Automatically Detect Running Ollama Port
def get_ollama_port():
    """Detects the active port on which Ollama is running."""
    try:
        result = subprocess.run(["lsof", "-i", "-P", "-n"], capture_output=True, text=True)
        for line in result.stdout.split("\n"):
            if "ollama" in line and "LISTEN" in line:
                parts = line.split()
                for part in parts:
                    if "127.0.0.1:" in part:
                        port = part.split(":")[-1]
                        return port
    except Exception as e:
        print(f"❌ ERROR: Failed to detect Ollama port. {e}")
    return None

# Get the active Ollama port
OLLAMA_PORT = get_ollama_port()
if not OLLAMA_PORT:
    print("❌ ERROR: Ollama is not running. Start it using 'ollama serve' in a separate terminal.")
    exit()
else:
    print(f"✅ Detected Ollama running on port {OLLAMA_PORT}")

# Set OLLAMA_HOST environment variable
os.environ["OLLAMA_HOST"] = f"http://127.0.0.1:{OLLAMA_PORT}"

# Step 2: Ensure Ollama is Running and Model Exists
try:
    response = ollama.list()
    available_models = [m["name"] for m in response["models"]]
    print("✅ Available models:", available_models)

    if "llama2:latest" not in available_models:
        print("❌ ERROR: Model 'llama2:latest' not found. Run: `ollama pull llama2:latest`")
        exit()
except Exception as e:
    print(f"❌ ERROR: Ollama is not responding on port {OLLAMA_PORT}. Start it using 'ollama serve'.")
    exit()

# Step 3: Load Test Samples
test_samples = [
    {
        "User Query": "SELECT AVG(pages) AS avg_pages, MAX(pages) AS max_pages, MIN(pages) AS min_pages FROM sandbox_app_library;",
        "Query Intent": "Aggregate statistics from the Library table",
        "Query Output": {"columns": ["avg pages", "max pages", "min pages"], "rows": [(575.12, 957, 111)]},
        "LLM Response": "The query output correctly aggregates the statistics from the Library table, providing the average, maximum, and minimum number of pages. The output is precise and meets the user’s intent."
    },
    {
        "User Query": "SELECT COUNT(*) FROM sandbox_app_users;",
        "Query Intent": "Count the total number of users in the Users table",
        "Query Output": {"columns": ["count"], "rows": [(1200,)]},
        "LLM Response": "The query output shows the total number of users, which is 1200. This correctly aligns with the intent of counting users in the table."
    }
]

# Step 4: Function to send a prompt to Ollama for evaluation
def evaluate_with_llm(user_query, query_output, llm_response, query_intent):
    """Evaluates an SQL chatbot response using a local LLM via Ollama."""

    prompt = f"""
    You are evaluating the correctness of an SQL chatbot response.
    Given the following:

    - **User Query**: {user_query}
    - **Expected Query Output**: {query_output}
    - **LLM Response**: {llm_response}
    - **User Intent**: {query_intent}

    Evaluate:
    1. **Accuracy** (0-5): How well does the LLM response match the expected query output?
    2. **Completeness** (0-5): Does the response explain the results clearly and correctly?

    Provide scores only in the format:
    Accuracy: X
    Completeness: Y
    """

    print("\n🔹 DEBUG: Sending prompt to LLM...")
    try:
        response = ollama.chat(model="llama2:latest", messages=[{"role": "system", "content": prompt}])
        result_text = response["message"]["content"]
        print("🔹 LLM Output:", result_text)

        # Step 5: Extract only numeric values (using regex to avoid parsing errors)
        accuracy_match = re.search(r'Accuracy:\s*(\d+)', result_text)
        completeness_match = re.search(r'Completeness:\s*(\d+)', result_text)

        if accuracy_match and completeness_match:
            accuracy_score = int(accuracy_match.group(1))
            completeness_score = int(completeness_match.group(1))
            return accuracy_score, completeness_score
        else:
            print("❌ ERROR: Failed to extract numeric scores. Skipping evaluation.")
            return None, None

    except Exception as e:
        print("❌ ERROR: Ollama evaluation failed:", e)
        return None, None

# Step 6: Evaluate all test samples
results = []
for sample in tqdm(test_samples, desc="Evaluating LLM responses"):
    accuracy, completeness = evaluate_with_llm(
        sample["User Query"], 
        sample["Query Output"], 
        sample["LLM Response"], 
        sample["Query Intent"]
    )

    if accuracy is None or completeness is None:
        print("\n⚠️ Skipping failed evaluation.")
        continue

    results.append({
        "User Query": sample["User Query"],
        "Accuracy": accuracy,
        "Completeness": completeness
    })

    time.sleep(1)  # Avoid overloading the model with requests

# Step 7: Convert results to DataFrame and Save
df_results = pd.DataFrame(results)

if not df_results.empty:
    csv_filename = "llm_evaluation_results.csv"
    df_results.to_csv(csv_filename, index=False)
    print(f"\n✅ Evaluation results saved to {csv_filename}")
else:
    print("\n❌ ERROR: No valid evaluations were recorded. Check logs for issues.")

# Step 8: Generate Graphs for Evaluation Metrics
if not df_results.empty:
    plt.figure(figsize=(10, 5))
    plt.hist(df_results["Accuracy"], bins=6, alpha=0.6, label="Accuracy", edgecolor='black')
    plt.hist(df_results["Completeness"], bins=6, alpha=0.6, label="Completeness", edgecolor='black')
    plt.xlabel("Score (0-5)")
    plt.ylabel("Number of Responses")
    plt.title("Distribution of Accuracy and Completeness Scores")
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("\n⚠️ Skipping graph generation as there are no valid results.")

✅ Detected Ollama running on port 56292
❌ ERROR: Ollama is not responding on port 56292. Start it using 'ollama serve'.


Evaluating LLM responses: 100%|██████████| 2/2 [00:00<00:00, 1342.39it/s]


🔹 DEBUG: Sending prompt to LLM...
❌ ERROR: Ollama evaluation failed: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

⚠️ Skipping failed evaluation.

🔹 DEBUG: Sending prompt to LLM...
❌ ERROR: Ollama evaluation failed: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

⚠️ Skipping failed evaluation.

❌ ERROR: No valid evaluations were recorded. Check logs for issues.

⚠️ Skipping graph generation as there are no valid results.


: 